In [ ]:
!pip install transformers==3.4.0
!pip install tensorflow
!pip install tflearn

     |████████████████████████████████| 1.3 MB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 34.8 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 2.9 MB 40.3 MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline
from pylab import rcParams
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
import tensorflow as tf   
import numpy as np
import tflearn           
import random
import json
from tensorflow.keras import layers

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
stemmer = LancasterStemmer()

PREDEFINE YOUR JSON FILE



```
# This is formatted as code
File : intents.json
{
  "intents":[
    {
      "tag":"LABEL1",
      "patterns":["QUESTION PATTERN 1","QUESTION PATTERN 2","QUESTION PATTERN N"],
      "responses":["ANSWER"]
    },  
    {
      "tag":"LABEL2",
      "patterns":["QUESTION PATTERN 1","QUESTION PATTERN 2","QUESTION PATTERN N"],
      "responses":["ANSWER"]
    }            
  ]
}
```



In [ ]:
with open("/content/intents.json") as json_data: 
    intents = json.load(json_data)      

In [ ]:
words=[]
documents = []
classes = []
ignore = ["?"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w = nltk.word_tokenize(pattern) 
        words.extend(w)
        documents.append((w, intent["tag"]))
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  
            

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words))) 
classes = sorted(list(set(classes)))

In [ ]:
training = []
output = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)
model = tflearn.DNN(net)
model.fit(train_x, train_y, n_epoch=1200, batch_size=8, show_metric=True) 
model.save("model.tflearn")

Training Step: 4799  | total loss: 0.01534 | time: 0.025s
| Adam | epoch: 1200 | loss: 0.01534 - acc: 1.0000 -- iter: 24/27
Training Step: 4800  | total loss: 0.01535 | time: 0.034s
| Adam | epoch: 1200 | loss: 0.01535 - acc: 1.0000 -- iter: 27/27
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.




---



In [ ]:
import pickle
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [ ]:
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open("intents.json") as json_data:
    intents = json.load(json_data)  

In [ ]:
model.load("model.tflearn") 

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [ ]:
context = {}
ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=True):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return random.choice(i['responses']), i['tag']
            results.pop(0)

INPUT



In [ ]:
r=""

In [ ]:
reply, tag = response(r)

In [ ]:
import torch

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
zero_shot_classifier = pipeline("zero-shot-classification", device = 0)

In [ ]:
zero_shot_classifier

In [ ]:
def zeroshot_classifier(input):
  result = zero_shot_classifier(
    sequences = input, 
    candidate_labels = ["INPUT CLASS LABEL 1","INPUT CLASS LABEL 2","INPUT CLASS LABEL 1"],
    multi_class= True
  )
  return result

PRE-DEFINE YOUR TAGS FOR SUPPORT

In [ ]:
if tag == "":
  multi_class_array = zeroshot_classifier(r)
else:
  print("")   


STORING THE 0-SHOT RESULTS

In [ ]:
sc=multi_class_array['scores']
lab=multi_class_array['labels']

In [ ]:
arr_hold = []

In [ ]:
i = 0
li = []
for x in range(0,2):
  li.append(lab[x])

GENERATING TOKENS USING DICTIONARIES



In [ ]:
d1 = {
    "UNIQUE KEY 1": "label trigger",
    "UNIQUE KEY 2": "label trigger",
    }
d2 = {
    "UNIQUE KEY 1": "label trigger",
    "UNIQUE KEY 2": "label trigger",
    }

In [ ]:
d = [d1,d2]
token = ''

In [ ]:
for i in d:
  for x, y in i.items():
    if y == li[0]:
      token = token + "" + x
      check_1 = x[0]
    if y == li[1]:
      token = token + ", " + x
      check_2 = x[0]

CHECKING THE INDEX OF THE TOKEN TO 

FOR EXAMPLE IF THE "UNIQUE KEY 1 = frontendteam1" and "UNIQUE KEY 2 = frontendteam2", the indexing extracts the "f" AND CHANNELIZES QUERIES RELATED TO FRONTEND TO THE FRONTEND CHANNEL SERVER VIA API 

In [ ]:
d_final = {
    "API TO CHANNEL 1": "index of UNIQUE KEY []",
    "API TO CHANNEL 2": "index of UNIQUE KEY []",
    "API TO CHANNEL 3": "index of UNIQUE KEY []"
}

In [ ]:
issue_token = ''

In [ ]:
for x, y in d_final.items():
  if y == check_1:
    #issue token 1
  if y == check_2:
    #issue token 2  

IF YOU NEED ANY HELP WITH API INTEGRATION WITH TELEGRAM, DISCORD, OR YOUR HOME SERVER/MARIX SERVER, HIT ME UP AT galadagautam911@gmail.com